In [3]:
!pip3 show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/ame/Library/Python/3.9/lib/python/site-packages
Requires: protobuf, keras, six, numpy, termcolor, astunparse, libclang, packaging, tensorboard, grpcio, h5py, gast, opt-einsum, setuptools, ml-dtypes, requests, absl-py, google-pasta, flatbuffers, typing-extensions, wrapt, tensorflow-io-gcs-filesystem
Required-by: tf_keras


In [4]:
# Minimal local predictor: works with Keras 3 or tf.keras
import json, numpy as np
from PIL import Image

# Try both loaders
_use_keras_loader = None
try:
    import keras  # Keras 3
    _KERAS_VERSION = getattr(keras, "__version__", "unknown")
except Exception:
    keras = None
    _KERAS_VERSION = None

import tensorflow as tf
from tensorflow.keras.preprocessing import image as tf_image
_TF_VERSION = tf.__version__

# Labels (adjust if yours differ)
CLASS_LABELS = {0: 'acne', 1: 'chickenpox', 2: 'monkeypox', 3: 'non-skin', 4: 'normal'}

def load_any_model(path: str):
    global _use_keras_loader
    if keras is not None:
        try:
            m = keras.saving.load_model(path, compile=False, safe_mode=False)
            _use_keras_loader = "keras"
            return m
        except Exception as e:
            print(f"[warn] Keras loader failed: {e}")
    try:
        from tensorflow import keras as tfk
        m = tfk.models.load_model(path, compile=False)
        _use_keras_loader = "tf.keras"
        return m
    except Exception as e:
        raise RuntimeError(f"Failed to load model with keras and tf.keras: {e}")

def preprocess_image(pil_image: Image.Image, size: int) -> np.ndarray:
    pil_image = pil_image.convert("RGB")
    img = pil_image.resize((size, size), Image.Resampling.LANCZOS)
    arr = tf_image.img_to_array(img, dtype=np.uint8)
    arr = arr / 255.0
    return np.expand_dims(arr, axis=0)  # (1, H, W, 3)

def predict_local(model_path: str, image_path: str, size: int = 300):
    model = load_any_model(model_path)
    pil_img = Image.open(image_path)
    x = preprocess_image(pil_img, size)

    preds = model.predict(x)
    if isinstance(preds, (list, tuple)):
        preds = preds[0]
    probs = preds[0] if preds.ndim == 2 else preds
    if probs.ndim != 1:
        raise ValueError(f"Expected 1D probabilities, got {probs.shape}")

    idx = int(np.argmax(probs))
    result = {
        "predicted_class": CLASS_LABELS.get(idx, str(idx)),
        "max_prob": float(np.max(probs)),
        "class_probabilities": {CLASS_LABELS.get(i, str(i)): float(round(p*100, 2)) for i, p in enumerate(probs)},
        "env": {"loader": _use_keras_loader, "keras_version": _KERAS_VERSION, "tensorflow_version": _TF_VERSION}
    }
    return result

In [9]:
# Set your local paths
IMAGE_PATH = "./uploads/sample.png"                # <-- change me
MODEL_PATH = "../models/model_10-0.92.keras"       # <-- change me
SIZE = 300                                         # per your model

out = predict_local(MODEL_PATH, IMAGE_PATH, SIZE)
print(json.dumps(out, indent=2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
{
  "predicted_class": "non-skin",
  "max_prob": 0.9989346861839294,
  "class_probabilities": {
    "acne": 0.009999999776482582,
    "chickenpox": 0.009999999776482582,
    "monkeypox": 0.07000000029802322,
    "non-skin": 99.88999938964844,
    "normal": 0.019999999552965164
  },
  "env": {
    "loader": "keras",
    "keras_version": "3.8.0",
    "tensorflow_version": "2.18.0"
  }
}
